**@author: James V. Talwar** <br>
*Created on January 12, 2023 01:22:21*

# Generate Mutation Best Rank Scores

**About:** This notebook operates as a scaffold for the deluxe skyscraper that is best rank scores. Here all SKCM missense mutations (those making there presence known in both DNA and RNA) are partitioned into their corresponding 8-11mers. Subsequently this notebook generates a bash script to run NetMHCPan4.1. After this script is generated, submitted, and the job completes, this notebook converts the results to mutation best rank scores for all alleles. The final section of this notebook is used to generate neoantigen burdens for TCGA individuals and mutation-specific (i.e., BRAF V600E) MHC-I presentation ability. *Let's begin...*

## 1. Import Packages; Load Data

In [1]:
import pandas as pd
import numpy as np
import sys, os
sys.path.append("/cellar/users/jtalwar/anaconda3/lib/python3.7/site-packages") #needed so NB will import Biopython
from Bio import SeqIO
from collections import defaultdict
import joblib
import time
import datetime
import tqdm

In [2]:
import logging
logging.getLogger().setLevel(logging.INFO)
logger = logging.getLogger()
console = logging.StreamHandler()
logger.addHandler(console)

Load in  patient mutations found in both DNA and RNA:

In [3]:
patientsToMutations = joblib.load("../GenotypeData/PatientToMutationMappings/PatientToMutations.pkl")

Load in TCGA RNA mutation calls which have mapping of mutations to transcripts:

In [4]:
melanomaRNAPath = "/cellar/users/andreabc/Data/TCGA/MAFs/masked_MAFs_by_disease/4b7a5729-b83e-4837-9b61-a6002dce1c0a/TCGA.SKCM.mutect.somatic.w_mut_IDS.w_RNA_counts.w_PHBR.w_VAF.maf"
rnaVAF = pd.read_csv(melanomaRNAPath, sep = "\t")

filteredRnaVAF = rnaVAF[["TCGA_Barcode", "Mutation_ID" , "Hugo_Symbol", "Transcript_ID", "Variant_Classification", "n_depth", "t_depth_x", "t_ref_count","t_alt_count_x", "readcount_total_depth", 'Reference_Allele_readcounts', 
                 "PHBR-I_score","PHBR-II_score", "VAF", "rank", "percentile_rank"]]
filteredRnaVAF = filteredRnaVAF[filteredRnaVAF["Variant_Classification"] == "Missense_Mutation"]


/cellar/users/jtalwar/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3343: DtypeWarning: Columns (87,88,98,118) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
mutationToTranscript = defaultdict(str)
for i, row in tqdm.tqdm(filteredRnaVAF[~filteredRnaVAF.Mutation_ID.isnull()].iterrows()):    
    mutation = row["Mutation_ID"].replace('"', '') #annoying... some strings are wrapped in strings - just why...
    if mutation in mutationToTranscript:
        if row["Transcript_ID"] != mutationToTranscript.get(mutation):
            logging.warning("Discrepancy for mutation {}. Previous {}; Current {}".format(mutation, mutationToTranscript.get(mutation),row["Transcript_ID"]))
    
    mutationToTranscript[mutation] = row["Transcript_ID"] 

197950it [00:15, 12722.63it/s]


In [6]:
transcripts = set([v for k,v in mutationToTranscript.items()])
logging.info("Number of ENST IDs: {}".format(len(transcripts)))

Number of ENST IDs: 17195


Load in TCGA summary DF and extract all unique HLA genotypes:

In [7]:
#get AI alleles
autoimmuneAlleles = set(pd.read_csv("../Data/AutoimmuneAlleles.tsv", sep = "\t", header = None)[0].tolist())
autoimmuneAlleles

{'HLA-B13:02',
 'HLA-B27:05',
 'HLA-B39:06',
 'HLA-B51:01',
 'HLA-B57:01',
 'HLA-C06:02',
 'HLA-C12:03'}

In [8]:
tcgaSummary = pd.read_csv("../GenotypeData/TCGA_Summary_With_BRAFV600E_Mutation_Status.tsv", index_col = 0, sep = "\t")

#Filter out individuals < 20 years old given increased likelihood of rare germline predisposing variants.
tcgaSummary = tcgaSummary[tcgaSummary.Age >= 20]

#Ensure AI Allele Status is correct
tcgaSummary["HasProtection"] = [(len(set(row["A1":"C2"]).intersection(autoimmuneAlleles)) > 0) for i,row in tcgaSummary.iterrows()]

In [9]:
tcgaHLA = set()
for i,row in tcgaSummary.iterrows():
    tcgaHLA = tcgaHLA.union(set(row["A1":"C2"]))

In [10]:
logging.info("There exist {} unique HLA genotypes in the SKCM TCGA.".format(len(tcgaHLA)))

There exist 118 unique HLA genotypes in the SKCM TCGA.


## 2. Extract Peptide Sequence from ENST IDs:

In [11]:
#The file ID mapping ENSTs to a number
peptide_map = np.load('/cellar/users/jtalwar/Data/Mappings/enst_to_file_number_dict.npy', 
                      allow_pickle=True).item()

In [12]:
logging.info("The number of missing ENST peptide mappings is: {}".format(len(transcripts) - len([peptide_map[el] for el in transcripts if el in peptide_map])))

The number of missing ENST peptide mappings is: 171


Define function to map to peptide sequence:

In [13]:
def ZhuLiDoTheThing(transcripts, basePath):
    getProteinSequence = defaultdict(str)
    missingIDs = set()
    for v in tqdm.tqdm(transcripts):
        try:
            findMe = os.path.join(basePath, peptide_map[v] + '.peptides')
            
            with open(findMe, 'r') as handle:
                sanityCheck = 0
                for entry in SeqIO.parse(handle, 'fasta'):
                    if 'ENST' + entry.id == v:
                        getProteinSequence[v] = str(entry.seq) #df index mapping to sequence
                        sanityCheck += 1 
                if sanityCheck > 1:
                    logging.warning("Hmmm... something fishy is afoot")
        except:
            missingIDs.add(v)
        
            
    logging.info("The total number of ENST IDs w/o a mapping to a peptide sequence is: {}".format(len(missingIDs)))
    return getProteinSequence

In [14]:
verrick = ZhuLiDoTheThing(transcripts = transcripts, basePath='/cellar/users/jtalwar/Data/Mappings/Peptides')

100%|██████████| 17195/17195 [06:13<00:00, 46.09it/s]
The total number of ENST IDs w/o a mapping to a peptide sequence is: 171


## 3. Generate all 8-11 mers (MHC-I) that overlap with mutation location:

In [15]:
#Input: Unmutated peptide sequence, window/k-mer length, position at which the amino acid substitution occurs (1-indexed), amino acid to change at given position
#Output: A set of mutated k-mers
def ExtractAllPeptides(peptide, window, position, unmutatedAA, swapsies): #Need to handle corner case where happens at beginning or end of peptide
    if len(peptide) < position:
        return {}, True
    
    if peptide[position - 1] != unmutatedAA: #it seems some are indeed 0-indexed so need to account for that as well...
        position += 1
    
    try:
        assert position < len(peptide), position < len(peptide)
        assert peptide[position - 1] == unmutatedAA,  peptide[position - 1] == unmutatedAA #sanity check to ensure things are correct
    
    except AssertionError as invalid:
        return {}, invalid
        
    #if peptide[position - 1] != unmutatedAA:
    #    raise ValueError("{} ; {}".format(peptide[position - 1], unmutatedAA))
    
    protein = peptide[:position-1] + swapsies + peptide[position:]
    
    if (len(peptide) != len(protein)): #another sanity check
        raise RuntimeWarning("Protein Assignment length != input peptide length ")
        
    kmers = set()
    for i in range(position-window, position): #len(protein) - window + 1):
        kmer = protein[i:i+window]
        if len(kmer) == window: #handle corner case at beginning and end
            kmers.add(kmer)
            
    return kmers, False #not invalid 

In [16]:
windows = [8,9,10,11]
mhc1Peptides = defaultdict(set) #mutation to corresponding peptides
invalidMutations = set()
for window in windows:
    counter = 0
    for k,v in tqdm.tqdm(mutationToTranscript.items()):
        counter += 1
        aaPosition = int(k.split("_")[-1][1:-1])
        unmutated = k.split("_")[-1][0]
        mutation = k.split("_")[-1][-1]
        
        tempo, invalid = ExtractAllPeptides(peptide = verrick[v], window = window, position = aaPosition, unmutatedAA = unmutated, swapsies = mutation)        
        mhc1Peptides[k] = mhc1Peptides[k].union(tempo) #invalid mutations are still mapped here, but as an empty set
        if invalid:
            invalidMutations.add(k)

100%|██████████| 181886/181886 [00:02<00:00, 66805.20it/s]


In [17]:
logging.info("Number of mutations that are inalid (i.e., wrong unmutated amino acid at position i or i -1, or mutation at an invalid position) is: {}".format(len(invalidMutations)))

Number of mutations that are inalid (i.e., wrong unmutated amino acid at position i or i -1, or mutation at an invalid position) is: 2415


In [18]:
logging.info("There are {} missense mutations w/ a greater than expected number of kmers due to different peptides produced from different transcripts.".format(len([(k,v) for k,v in mhc1Peptides.items() if len(v) > 8+9+10+11]))) #These are result of those with different peptides produced by different transcripts

There are 0 missense mutations w/ a greater than expected number of kmers due to different peptides produced from different transcripts.


Save all mutated peptides (8-11 mers):

In [19]:
with open('./allSkcmNeopeptides.peptides', "w") as peptides:
    for k,v in mhc1Peptides.items():
        for val in v:
            peptides.write(val + "\n")

In [20]:
allMHC1Peptides = np.loadtxt('./allSkcmNeopeptides.peptides', dtype=str)

In [21]:
mhc1Peps = set(allMHC1Peptides)

In [22]:
logging.info("There are {} unique peptides to be evaluated by NetMHCPan4.1".format(len(mhc1Peps)))
savePath = './Mutations.peptides'
np.savetxt(savePath, list(mhc1Peps), delimiter='\n', fmt='%s')

There are 6689577 unique peptides to be evaluated by NetMHCPan4.1


In [23]:
del(allMHC1Peptides)


## 3. Generate a bash script to run NetMHCPan:

In [24]:
mhc1Alleles = np.loadtxt('/cellar/users/andreabc/Data/refs/hla/mhc_i_types.txt', dtype=str)
print('{} unique MHC-I Alleles'.format(len(mhc1Alleles)))

2915 unique MHC-I Alleles


Subset down to only TCGA alleles:

In [25]:
#If certain jobs finish and others time out filter down to those that didn't finish and re-run; Uncomment muted code below in this case
finishedAlleles = set() 

'''
finishedFilesBasePath = os.path.join(os.getcwd(), "mhc1Files")
finishedFiles = [os.path.join(finishedFilesBasePath, el) for el in os.listdir(finishedFilesBasePath) if ".xlsoutput" in el]

for file in finishedFiles:
    f = open(file, 'r')
    for line in f:
        finishedAlleles = finishedAlleles.union(set([el for el in line.split("\t") if el != "" and el != "\n"]))
        break
    f.close()
'''

'\nfinishedFilesBasePath = os.path.join(os.getcwd(), "mhc1Files")\nfinishedFiles = [os.path.join(finishedFilesBasePath, el) for el in os.listdir(finishedFilesBasePath) if ".xlsoutput" in el]\n\nfor file in finishedFiles:\n    f = open(file, \'r\')\n    for line in f:\n        finishedAlleles = finishedAlleles.union(set([el for el in line.split("\t") if el != "" and el != "\n"]))\n        break\n    f.close()\n'

In [26]:
mhc1Alleles = [el for el in mhc1Alleles if el in tcgaHLA and el not in finishedAlleles]
logging.info("All tcga genotypes extracted? {}".format(len(set(mhc1Alleles)) == len(tcgaHLA)))

All tcga genotypes extracted? True


In [27]:
len(mhc1Alleles)

118

In [28]:
def ChunkyKong(allAlleles, chunkSize):
    for i in range(0, len(allAlleles), chunkSize):
        yield allAlleles[i:i + chunkSize]

forRunningInScript1 = [','.join(x) for x in ChunkyKong(mhc1Alleles, 3)] #3 alleles in each job
print('{} MHC-I jobs'.format(len(forRunningInScript1)))


40 MHC-I jobs


<div class="alert alert-block alert-warning">
<b>Notes on running NetMHCPan script generated by method below:</b>
<ul>
    <b><li>When generating the script provide new paths to locations of all relevant files</li></b>
    <b><li>Ensure when running script  have err,out, mhc1Files folders in same place as the script</li></b>
    <b><li>Script generated will run 50 jobs in parallel with each job corresponding to 3 (i.e., chunkSize defined above) MHC alleles</li></b>
</div>

In [29]:
#inputFiles == peptides here
def CreateClusterScript(allele_list, pathToScript, inputFiles, outDir, whichPan, scriptName = 'run_netmhcpan4_1.sh'):
    new_script_file = os.path.join(pathToScript, scriptName)
    
    logging.info("Writing bash script to the following:" + new_script_file)

    with open(new_script_file, 'w') as out_file:
        out_file.write("#!/bin/bash\n")
        out_file.write("#SBATCH --output={}/out/%A.%x.%a.out\n".format(pathToScript)) # Out file path and file names
        out_file.write("#SBATCH --error={}/err/%A.%x.%a.err\n".format(pathToScript))  # Error file path and file names
        out_file.write("#SBATCH --array=1-{}%50\n".format(len(allele_list)))          # Array job --> 50; Change as needed
        out_file.write("#SBATCH --mem=25G\n")                                        #
        out_file.write("#SBATCH -t 9-03:31:13\n\n")                                  # Time-limit before cluster kills job
    
        out_file.write("alleles=({})\n".format(" ".join(allele_list))) #Write the MHC-1/2 alleles
        out_file.write("allele_string=${alleles[$SLURM_ARRAY_TASK_ID - 1]}\n\n") #slurm array job for parallelization
        
        out_file.write("start_time=$SECONDS\n")
        
        out_file.write('NETMHCPAN={}\n'.format(whichPan))
        
        out_file.write('INPUT_FILE={}\n'.format(inputFiles))
        out_file.write('OUTPUT_DIR={}\n'.format(outDir))
                
        out_file.write("date\n")
        out_file.write("hostname\n\n")
        
        # Run netmhcpan
        out_file.write('$NETMHCPAN -a $allele_string -inptype 1 -f $INPUT_FILE -xls -xlsfile $OUTPUT_DIR/$SLURM_ARRAY_TASK_ID.xlsoutput \n\n')
        out_file.write("date\n")
        out_file.write("elapsed=$(( SECONDS - start_time ))\n")
        out_file.write('eval "echo Total Script Run Time/Elapsed time: $(date -ud "@$elapsed" +')
        out_file.write("'$((%s/3600/24)) days %H hr %M min %S sec'" + ')"')       
        

'\n    with open(new_script_file, \'w\') as out_file:\n        out_file.write("#!/bin/bash\n")\n        out_file.write("#SBATCH --output={}/out/%A.%x.%a.out\n".format(pathToScript)) # Out file path and file names\n        out_file.write("#SBATCH --error={}/err/%A.%x.%a.err\n".format(pathToScript))  # Error file path and file names\n        out_file.write("#SBATCH --array=1-{}%50\n".format(len(allele_list)))          # Array job --> 50; Change as needed\n        out_file.write("#SBATCH --mem=25G\n")                                        #\n        out_file.write("#SBATCH -t 9-03:31:13\n\n")                                  # Time-limit before cluster kills job\n    \n        out_file.write("alleles=({})\n".format(" ".join(allele_list))) #Write the MHC-1/2 alleles\n        out_file.write("allele_string=${alleles[$SLURM_ARRAY_TASK_ID - 1]}\n\n") #slurm array job for parallelization\n        \n        out_file.write("start_time=$SECONDS\n")\n        \n        out_file.write(\'NETMHCPAN={}

In [30]:
CreateClusterScript(allele_list=forRunningInScript1, pathToScript = os.getcwd(), inputFiles=os.path.join(os.getcwd(), 'Mutations.peptides'), outDir = os.path.join(os.getcwd(),'OtherMHC1Files'), whichPan ='/cellar/users/andreabc/nrnb/software/netMHCpan-4.1/netMHCpan')

Writing bash script to the following:/cellar/users/jtalwar/CleanedCode/MelMHC/Preprocessing/run_netmhcpan4_1.sh


## 5. Converting NetMHCPan Results to Best Rank Score Affinity Profile Matrices

**Extracting Peptides (rows) and alleles (columns)**:

In [31]:
'''
INPUT(s) 
 1) theThing: List of paths to NetMHCPan output files 
 2) savePath: String corresponding to the path and file name want to write file (ex: /cellar/users/jtalwar/projects/melanoma/VEST_Results/ or os.path.join(os.getcwd(), "Compiled/"))
 3) dos: boolean indicating whether results are MHC-II or MHC-I (they have different file formats) (default = False --> MHC-I)

OUTPUT(s):
 None --> This function takes in the inputs and merges the results into a single dataframe and writes that dataframe to the savePath with the name PeptideHLADF.csv
'''

def BolinDoTheThing(theThing, savePath, dos = False):
    assert os.path.exists(savePath)
    
    savePath = os.path.join(savePath, "PeptideHLADF.csv")
    numHLAAlleleFiles = len([el for i in range(len(theThing)) for el in os.listdir(theThing[i]) if el.endswith(".xlsoutput")])
    
    logging.info(f"What Thing? Oh I see - okay doing the thing... I'll be sure to put the results in {savePath}\n")
    
    mapEverything = defaultdict(lambda: defaultdict(float))
    soItBegins = time.time()
    howManyFilesProcessed = 0
    
    for directory in theThing:
        for file in tqdm.tqdm(os.listdir(directory)):
            if not file.endswith(".xlsoutput"): #occasionaly there can be an ipynb checkpoints folder/file here 
                continue 

            f = open(os.path.join(directory, file), 'r')
            alleles = []
            for line in f:
                alleles = [el for el in line.split("\t") if el != "" and el != "\n"]
                break
            f.close()

            temp = pd.read_csv(os.path.join(directory, file), sep ="\t", skiprows=1) 

            if not dos:
                correspondingRanks = ['EL_Rank'] + ['EL_Rank.' + str(i) for i in range(1, len(alleles))] #['EL_Rank','EL_Rank.1','EL_Rank.2','EL_Rank.3','EL_Rank.4']# ranks are in the same order of the alleles --> list comp would make this generalizable...
                
            else:
                correspondingRanks = ['Rank'] + ['Rank.' + str(i) for i in range(1, len(alleles))] #['Rank','Rank.1','Rank.2','Rank.3','Rank.4']
            
            happyTogether = dict(zip(correspondingRanks, alleles)) #(EL_)Rank. column to allele
            
            for k,v in happyTogether.items():
                mapEverything[v] = dict(zip(temp.Peptide, temp[k]))
            
            '''
            for i in temp.index:
                for k,v in happyTogether.items(): #better way is to zip as a dict and then for each allele in an outer dict assign this dict to the allele
                    mapEverything[v][temp.loc[i, "Peptide"]] = temp.loc[i, k] 
            '''
            
            howManyFilesProcessed += 1
            if howManyFilesProcessed % 10 == 0:
                if not dos:
                    logging.info("Finished: {} of {} allele files".format(howManyFilesProcessed, numHLAAlleleFiles))
                else:
                    logging.info("Finished: {} of {} allele files".format(howManyFilesProcessed, numHLAAlleleFiles))

    elapsed = time.time() - soItBegins
    logging.info("Elapsed Time: {}".format(datetime.timedelta(seconds = elapsed)))       
        
    logging.info('Converting to a DataFrame:')
    
    mappingDataFrame = pd.DataFrame(mapEverything)
    logging.info('Saving to save path...\n')
    mappingDataFrame.to_csv(savePath)
    logging.info('You did it kid... You did the thing!!!')

Generate the Peptide-HLA composite dataframe:

In [32]:
BolinDoTheThing(theThing = netMHCPanScoreFileDirectories, savePath = os.path.join(os.path.join(os.getcwd(), "Compiled/PeptideHLADF.csv")))

Read in composite file(s) generated:

In [33]:
peptideToHLAMapping = pd.read_csv(os.path.join(os.path.join(os.getcwd(), "Compiled/PeptideHLADF.csv")))
peptideToHLAMapping.head()

,Unnamed: 0,HLA-B41:02,HLA-B42:01,HLA-B44:02,HLA-C12:02,HLA-C12:03,HLA-C14:02,HLA-B67:01,HLA-B81:01,HLA-C01:02,...,HLA-B35:08,HLA-A30:04,HLA-A31:01,HLA-A32:01,HLA-A02:06,HLA-A02:17,HLA-A03:01,HLA-A26:01,HLA-A26:08,HLA-A29:01
0,SRLHPGGHP,18.7637,55.2778,21.5238,51.4286,50.0,41.2500,35.4167,30.4545,72.5000,...,68.3333,41.6180,47.0000,64.0,59.8276,63.0,41.4000,51.8182,50.9091,54.0000
1,EDLLCYSKL,3.0468,24.3814,6.5789,56.0000,42.5,53.3333,22.3235,22.8966,45.0000,...,23.3704,38.0183,57.3333,47.0,39.8696,34.5,68.7500,21.0213,21.5185,38.7143
2,NPPRFSQS,58.0000,11.0802,58.7500,51.4286,45.0,37.6667,10.9328,26.8824,24.5926,...,18.1935,92.3810,83.7500,85.0,80.0000,80.0,80.0000,68.7500,71.0000,56.2500
3,ETTLWAWFQ,87.5000,70.0000,53.0000,61.6667,60.0,72.5000,71.6667,53.5714,70.0000,...,36.0000,54.5814,15.8690,44.4,61.7500,75.0,32.7273,6.4304,5.4643,30.2143
4,MRQKIAEKTS,45.8571,69.4444,57.5000,65.0000,67.5,72.5000,60.0000,65.0000,70.0000,...,65.0000,58.0719,76.0000,77.5,63.2500,75.0,71.6667,75.0000,87.5000,85.0000


Converting from peptides to mutation level best rank scores:

In [34]:
#SANITY CHECK:
logging.info("All mutations with no mapped peptides in mhc1Peptides are invalid mutations: {}".format(set([k for k,v in mhc1Peptides.items() if len(v) == 0]) == invalidMutations))

All mutations with no mapped peptides in mhc1Peptides are invalid mutations: True


In [35]:
'''
INPUT(s):
 1) compendium: Peptide to HLA (pandas dataframe)
 2) websters: Dictionary mapping each mutation to its corresponding k-mers
 
OUTPUT(s):
 1) Best rank score affinity profile dataframe
'''
def GetBRScores(compendium, websters):
    soYouWantToBeAPokemonMaster = defaultdict(lambda: defaultdict(float))
    for mutation in tqdm.tqdm(websters):
        if len(websters[mutation]) == 0: #These mutations could not be called due to VEST-transcript issues (e.g., Position of mutation > length of the mutation)
            continue
        subframe = compendium[compendium['Unnamed: 0'].isin(websters.get(mutation))]
        for allele in compendium.columns[1:]: #skips the unnamed: 0 of peptides
            soYouWantToBeAPokemonMaster['M_' + mutation][allele] = min(subframe[allele])
            
    return pd.DataFrame(soYouWantToBeAPokemonMaster)

In [40]:
allMutsBR = GetBRScores(peptideToHLAMapping, mhc1Peptides).T
#allMutsBR.to_csv("../Data/AllMutationsBRMHC1.tsv", sep = "\t") #Save all compiled mutation BR scores 
allMutsBR.head()

,HLA-B41:02,HLA-B42:01,HLA-B44:02,HLA-C12:02,HLA-C12:03,HLA-C14:02,HLA-B67:01,HLA-B81:01,HLA-C01:02,HLA-B51:08,...,HLA-B35:08,HLA-A30:04,HLA-A31:01,HLA-A32:01,HLA-A02:06,HLA-A02:17,HLA-A03:01,HLA-A26:01,HLA-A26:08,HLA-A29:01
M_M_CAMTA1_S287L,5.3307,0.9563,9.4911,0.7698,0.3099,2.3514,1.7232,1.2152,0.7327,3.2909,...,0.8195,6.2557,0.0885,5.9258,3.7998,2.7453,0.9787,4.4246,2.3806,4.5695
M_M_C8B_R121Q,8.8238,1.7444,10.5648,13.8580,21.7200,20.2034,0.9338,1.4433,11.7556,8.4536,...,6.7145,8.3721,18.1484,10.0267,1.4824,9.5160,22.5000,13.4078,13.3066,14.9706
M_M_ARHGAP29_E549K,5.1769,3.8730,1.5164,4.8405,3.5412,4.8571,6.2576,6.9432,1.6890,19.8138,...,17.9740,10.2382,0.6652,7.0347,17.8205,18.1778,2.1259,4.0422,4.4006,12.2249
M_M_LRRC39_R231Q,3.1753,7.4188,6.3424,3.4966,6.2062,4.8240,6.7726,3.9990,3.2382,2.4776,...,11.3592,6.8742,16.9696,1.9583,0.9093,0.3103,13.3913,15.3669,12.0200,8.5043
M_M_NTNG1_D31G,12.8264,0.3776,8.4378,0.8774,1.2729,1.3785,0.4073,0.1547,7.3177,0.8833,...,1.8807,1.7532,4.3461,8.3837,7.3821,1.5785,0.6838,7.0597,4.5759,0.8465


In [41]:
#Doubled the "M_" prefix - remove that
allMutsBR.index = [el[2:] for el in allMutsBR.index]  

**Sanity Check**: Quality control - ensure best rank scores are consistent with 215 driver mutations

In [42]:
#Load in and filter driver mutation best rank scores
driverAffinityProfiles = pd.read_csv("../Data/Candidate_Drivers_BR_MHC1.tsv", sep = "\t", index_col = 0)
driverAffinityProfiles = driverAffinityProfiles.T[driverAffinityProfiles.columns.isin(allMutsBR.columns)].T
driverAffinityProfiles.sort_index(inplace = True)
driverAffinityProfiles.sort_index(inplace = True, axis = 1)
driverAffinityProfiles.head()

,HLA-A01:01,HLA-A01:02,HLA-A01:03,HLA-A02:01,HLA-A02:03,HLA-A02:05,HLA-A02:06,HLA-A02:17,HLA-A03:01,HLA-A03:02,...,HLA-C12:02,HLA-C12:03,HLA-C14:02,HLA-C15:02,HLA-C15:05,HLA-C16:01,HLA-C16:02,HLA-C16:04,HLA-C17:01,HLA-C18:01
mutations,,,,,,,,,,,,,,,,,,,,,
M_ABI1_Y37H,3.8008,4.6856,4.9177,8.0503,6.9839,2.5120,4.6197,9.3652,2.0282,1.8338,...,10.5354,4.7839,12.4548,5.6839,6.4017,3.3460,7.8427,3.8659,10.4156,8.3380
M_ADAM18_M143I,10.5601,5.0832,10.0261,2.3964,2.3015,1.1458,0.9416,1.4517,2.7471,2.4982,...,11.8173,7.6707,1.7467,4.4450,3.7725,11.7972,9.2221,11.6242,4.9811,0.4505
M_ADAM28_E340K,4.0003,3.9029,3.3626,9.2953,6.0302,2.9363,5.3336,6.9515,0.0083,0.0090,...,0.0836,0.1276,1.5563,1.0952,0.8731,0.0717,0.1126,0.0754,0.9658,2.5878
M_ADAM7_R225Q,12.7416,8.4973,11.0714,0.5214,0.9849,1.1599,0.4053,0.4698,3.2920,4.8739,...,0.6218,2.2604,3.9682,4.3695,6.6497,5.4039,4.7784,3.3590,3.2598,4.9425
M_ADCYAP1R1_R413Q,4.3820,2.8423,3.2814,2.7138,1.3210,1.3445,1.2895,2.1165,1.2018,1.2715,...,0.8915,1.3902,4.5561,1.7192,2.6334,2.0356,1.3861,1.1703,3.2527,2.5307


In [43]:
driverAllMuts = allMutsBR[allMutsBR.index.isin(driverAffinityProfiles.index)]
driverAllMuts.sort_index(inplace = True)
driverAllMuts.sort_index(inplace = True, axis = 1)
driverAllMuts.head()

/cellar/users/jtalwar/miniconda3/envs/Base/lib/python3.7/site-packages/pandas/core/frame.py:6402: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  key,


,HLA-A01:01,HLA-A01:02,HLA-A01:03,HLA-A02:01,HLA-A02:03,HLA-A02:05,HLA-A02:06,HLA-A02:17,HLA-A03:01,HLA-A03:02,...,HLA-C12:02,HLA-C12:03,HLA-C14:02,HLA-C15:02,HLA-C15:05,HLA-C16:01,HLA-C16:02,HLA-C16:04,HLA-C17:01,HLA-C18:01
M_ABI1_Y37H,3.8008,4.6856,4.9177,8.0503,6.9839,2.5120,4.6197,9.3652,2.0282,1.8338,...,10.5354,4.7839,12.4548,5.6839,6.4017,3.3460,7.8427,3.8659,10.4156,8.3380
M_ADAM18_M143I,10.5601,5.0832,10.0261,2.3964,2.3015,1.1458,0.9416,1.4517,2.7471,2.4982,...,11.8173,7.6707,1.7467,4.4450,3.7725,11.7972,9.2221,11.6242,4.9811,0.4505
M_ADAM28_E340K,4.0003,3.9029,3.3626,9.2953,6.0302,2.9363,5.3336,6.9515,0.0083,0.0090,...,0.0836,0.1276,1.5563,1.0952,0.8731,0.0717,0.1126,0.0754,0.9658,2.5878
M_ADAM7_R225Q,12.7416,8.4973,11.0714,0.5214,0.9849,1.1599,0.4053,0.4698,3.2920,4.8739,...,0.6218,2.2604,3.9682,4.3695,6.6497,5.4039,4.7784,3.3590,3.2598,4.9425
M_ADCYAP1R1_R413Q,4.3820,2.8423,3.2814,2.7138,1.3210,1.3445,1.2895,2.1165,1.2018,1.2715,...,0.8915,1.3902,4.5561,1.7192,2.6334,2.0356,1.3861,1.1703,3.2527,2.5307


In [44]:
logging.info("Dimensions consistent? {}".format(driverAllMuts.shape == driverAffinityProfiles.shape))
logging.info("Indexes consistent? {}".format((driverAllMuts.index == driverAffinityProfiles.index).all()))
logging.info("Columns consistent? {}".format((driverAllMuts.columns == driverAffinityProfiles.columns).all()))

Dimensions consistent? True
Indexes consistent? True
Columns consistent? True


In [45]:
logging.info("Mutation Best Rank Consistency? {}".format(driverAllMuts.round(11).equals(driverAffinityProfiles.round(11)))) #Consistent to 11 decimal places

Mutation Best Rank Consistency? True


## 6. Computing and Storing Strong and Weak Binding Neoantigen Burdens for Individuals 

 - Strong: PHBR < 0.5
 - Weak: PHBR < 2
 
**Define method for calculating PHBR**:

In [46]:
'''
INPUT(s):
 1) affinityProfile: A pandas dataframe of mutation (rows) and HLA-alleles (columns; Class I alleles) with best rank scores
 2) mutation: Str (or any valid index instance) corresponding to a particular row/mutation in affinityProfiles (if it was capable of being computed: i.e., it was a valid mutation)
 3) alleles: List of an individuals 6 MHC-I/HLA alleles 
 
OUTPUT(s):
 1) phbr: An individuals PHBR score for the given mutation (float)
 2) mutationValid: Whether the given mutation existed in affinityProfiles (bool). If using the full mutation BR affinityProfiles, this corresponds 
    to whether a mutation was valid/capable of having BR scores computed. 
    
About: Method for calculating PHBR-I scores
'''
def CalcPHBR(affinityProfile, mutation, alleles):
    assert len(alleles) == 6

    mutationValid = mutation in affinityProfile.index
    
    if not mutationValid:
        return None, mutationValid
    
    inverseBestRankScore = 0
    for allele in alleles:
        inverseBestRankScore += 1/affinityProfile.loc[mutation, allele]
    
    phbr = 6/inverseBestRankScore
    
    return phbr, mutationValid

**Compute neoantigen burdens:**

In [47]:
neoantigenBurden = defaultdict(lambda: defaultdict(int)) #Keys TCGA-_ _ _ _:{StrongBinding_PHBR-I_0.5 WeakBinding_PHBR-I_2 TotalMutations ValidMutations InvalidMutations}
strongBindThresh = 0.5
weakBindThresh = 2

for k,v in tqdm.tqdm(patientsToMutations.items()):
    if k not in tcgaSummary.index: #Get for filtered set (>= 20 years of age of diagnosis, not MSI etc.)
        continue
        
    patientAlleles = list(tcgaSummary.loc[k, "A1":"C2"])
    neoantigenBurden[k]["TotalMutations"] = len(v) #patient specific number of mutations in both DNA and RNA
    
    for mutation in v:
        phbr, valMut = CalcPHBR(affinityProfile = allMutsBR, mutation = mutation, alleles = patientAlleles) 
        
        if not valMut:
            neoantigenBurden[k]["InvalidMutations"] += int(not valMut)
            continue
            
        neoantigenBurden[k]["ValidMutations"] += int(valMut)
        neoantigenBurden[k]["StrongBinding_PHBR-I_0.5"] += int(phbr < strongBindThresh)
        neoantigenBurden[k]["WeakBinding_PHBR-I_2"] += int(phbr < weakBindThresh)
        
    if "InvalidMutations" not in neoantigenBurden[k]:
        neoantigenBurden[k]["InvalidMutations"] = 0 #Handles issue of NaNs/None when convert to DF

100%|██████████| 464/464 [00:11<00:00, 41.41it/s]


**Compute ability to present BRAF V600E at weak and strong binding thresholds:**

In [48]:
for k,v in patientsToMutations.items():
    if k not in tcgaSummary.index: #Get for filtered set (>= 20 years of age of diagnosis, not MSI etc.)
        continue 
        
    patientAlleles = list(tcgaSummary.loc[k, "A1":"C2"])
    phbr, valMut = CalcPHBR(affinityProfile = allMutsBR, mutation = "M_BRAF_V600E", alleles = patientAlleles) 
    
    assert valMut
    
    neoantigenBurden[k]["WeakV600EPresentation"] = int(phbr < weakBindThresh) 
    neoantigenBurden[k]["StrongV600EPresentation"] = int(phbr < strongBindThresh)

**Save Results:**

In [50]:
pd.DataFrame(neoantigenBurden).T[["StrongBinding_PHBR-I_0.5", "WeakBinding_PHBR-I_2", "InvalidMutations", "ValidMutations", "TotalMutations", "StrongV600EPresentation", "WeakV600EPresentation"]].to_csv("../GenotypeData/NeoantigenBurdens_SKCM_TCGA.tsv", sep = "\t")